## Прогнозирование времени дожития пациентов с боковым амиотрофическим склерозом на основее графа водородных связей фермента SOD1.
  
<h3>Задачи работы</h3>
<blockquote>Разработка метода вычисления характеристик графа водородных связей, который позволит отобрать оптимальные наборы характеристик и оптимальные способы расчета, дающие лучший результат прогнозирования. </blockquote>
<blockquote>Построение и исследование регрессионных моделей, связывающих характеристики графа водородных связей, ассоциированных с БАС с дожитием пациентов, являющихся носителями данных мутаций. Предсказание дожития пациентов с мутациями в SOD1.</blockquote>
<blockquote>Анализ результатов построенной регрессионной модели и выделение участков фермента SOD1, влияющих на продолжительность жизни.</blockquote>

<h4>Также рассматривалось прогнозирование времени дожиия пациентов с БАС с использованием метода PLS</h4>
Описание работы представлено в данном сборнике на 248-251 страницах.

https://lomonosov-msu.ru/file/event/6692/eid6692_attach_404139f68c88129a8743911f358bebedf89ab2ea.pdf

In [15]:
!python -V

Python 3.10.4


<h3>Данные хранятся по адресу: https://disk.yandex.ru/d/xl11LO2vfM7juw</h3>

In [3]:
import pandas as pd
path = 'graph_value.csv'
input_data = pd.read_csv(path, sep=',')

<h2>Информация о данных:</h2>
В качестве наименования каждого столбца в данных выступают пациенты и соответствующие им идентификаторы. Последняя строка – это время дожития пациентов с БАС, измеряемое в годах. В первом столбце содержится информация о взаимодействие донора и акцептора.  В остальных столбцах содержится вес, соответствующий стабильности определенной водородной связи. Всего рассматривается 182 связи между донором и акцептором. 

Донор и акцептор – аминокислоты, состоящие из: позиции аминокислотного остатка, имени субъединицы, названии атома. Таким образом, рассматривая пример донора с наименованием 6.A@N: цифра 6 будет являться позицией аминокислотного остатка, буква А после точки будет являться именем субъединицы и буква N, стоящая после знака собачки будет являться названием атома. 

Информация о структуре фермента SOD1 для 72 пациентов имеет 182 признака. Такая структура данных является проблемой так как количество объясняющих переменных гораздо больше чем количество наблюдаемых переменных. По этой же причине высока вероятность возникновения эффекта мультиколлинеарности. 

Мультиколлинеарность – ситуация, когда две или более объясняющие переменные линейно зависимы, что может привести к малоэффективному оцениванию параметров. Увеличивает вероятность возникновения данного эффекта тот факт, что для всех пациентов рассматривается один и тот же белок, у которого водородные связи могут не сильно отличаться друг от друга.

Учитывая все особенности данных, рассматривается построение регрессионных моделей на основе графовых структур, а не на основе информации о стабильности водородных связей аминокислоты.

In [5]:
input_data

,Unnamed: 0,A4T,A4V,A4S,C6G,C6F,V7E,L8Q,G10V,G12A,...,A140G,G141E,L144F,L144S,A145T,C146R,V148G,V148I,I149T,I151T
0,4.A@N-20.A@O,0.821000,0.851000,0.817,0.873,0.860,0.848000,0.899,0.901,0.903,...,0.893,0.876,0.858000,0.921,0.879,0.894,0.832000,0.832000,0.848,0.880
1,5.A@N-150.A@O,0.906000,0.974000,0.901,0.957,0.921,0.940000,0.916,0.952,0.943,...,0.936,0.942,0.924000,0.937,0.943,0.958,0.963000,0.931000,0.942,0.948
2,6.A@N-18.A@O,0.928000,0.899000,0.926,0.937,0.944,0.901000,0.884,0.928,0.936,...,0.913,0.892,0.922000,0.930,0.900,0.905,0.928000,0.943000,0.929,0.944
3,8.A@N-16.A@O,0.836000,0.820000,0.840,0.869,0.845,0.857000,0.852,0.761,0.799,...,0.801,0.844,0.845000,0.799,0.850,0.822,0.839000,0.825000,0.781,0.754
4,6.A@O-18.A@N,0.908000,0.858000,0.891,0.812,0.908,0.911000,0.903,0.906,0.901,...,0.902,0.874,0.906000,0.876,0.886,0.913,0.926000,0.918000,0.868,0.867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,46.B@NH2-124.B@OD2,0.000000,0.000000,0.000,0.000,0.000,0.000000,0.000,0.000,0.000,...,0.000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.000
179,46.A@NH1-139.A@O,0.000000,0.000000,0.000,0.000,0.000,0.000000,0.000,0.000,0.000,...,0.000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.000
180,48.B@NE2-61.B@O,0.000000,0.000000,0.000,0.000,0.000,0.000000,0.000,0.000,0.000,...,0.000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.000
181,65.A@OG-67.A@O,0.000000,0.000000,0.000,0.000,0.000,0.000000,0.000,0.000,0.000,...,0.000,0.000,0.000000,0.000,0.000,0.000,0.000000,0.000000,0.000,0.000


<h2>Построение графа</h2>

Так как водородные связи фермента SOD1 это взаимодействие между донором и акцептором, то по существующим данным можно построить и рассматривать граф. Граф можно построить тремя различными способами.

In [33]:
from 

ImportError: attempted relative import with no known parent package